In [ ]:
import os.path
import pandas as pd
import math

from bs4 import BeautifulSoup
from lxml import etree
import requests

HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
            (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',\
            'Accept-Language': 'en-US, en;q=0.5'})



# df = pd.read_csv('breaks.csv', ignore_index)

df = pd.DataFrame({
    'break_name': []
    , 'break_url': []
    , 'time': []
    , 'rating': []
    , 'period': []
    , 'wind': []
    , 'wind_state': []
    , 'wave_height': []
    , 'wave_direction': []
    , 'latitude': []
    , 'longitude': []
})


pages_df = pd.read_csv('pages.csv')

if len(pages_df) < 3000:
    pages_df = pd.DataFrame({'break_name': [], 'url': [], 'page_number': []})

    c = True
    while c:
        print('Page {}'.format(page_num) )
        URL = 'https://www.surf-forecast.com/breaks?page={}'.format(5)
        webpage = requests.get(URL, headers=HEADERS)
        soup = BeautifulSoup(webpage.content, "html.parser")
        breaks = soup.find_all('table', class_='list_table')[0].find_all('a')

        if len(breaks) == 0
            c = False
            continue

        for surf_spot in breaks:
            pages_df = pd.concat([pages_df, pd.DataFrame({
                'break_name': [surf_spot.text]
                , 'url': [surf_spot['href']]
                , 'page_number': [int(page_num)]
            })], ignore_index=True)

def scrape(row):
    cur_break_df = pd.DataFrame({
            'break_name': []
            , 'break_url': []
            , 'time': []
            , 'rating': []
            , 'period': []
            , 'wind': []
            , 'wind_state': []
            , 'wave_height': []
            , 'latitude': []
            , 'longitude': []
        })

    URL = row['url'] + '/forecasts/latest'

    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")
    dom = etree.HTML(str(soup))


    ratings = soup.find_all('tbody')[1].find_all('tr')[2].find_all('td')
    times = soup.find_all('tbody')[1].find_all('tr')[1].find_all('td')
    periods = soup.find_all('tbody')[1].find_all('tr')[5].find_all('td')    
    winds = soup.find_all('tbody')[1].find_all('tr')[8].find_all('td')
    wave_heights = soup.find_all('tbody')[1].find_all('tr')[4].find_all('td')
    wind_states = soup.find_all('tbody')[1].find_all('tr')[9].find_all('td')
    
    try:
        latitude = soup.find_all('span', class_='latitude')[0]['title']
        longitude = soup.find_all('span', class_='longitude')[0]['title']
    except:
        latitude = None 
        longitude = None

    for time, rating, period, wind, wind_state, wave_height in zip(
            times, ratings, periods, winds, wind_states, wave_heights):
        cur_break_df = pd.concat([cur_break_df, pd.DataFrame({
            'break_name': [row['break_name']]
            , 'break_url': [row['url']]
            , 'time': [time.text]
            , 'rating': [rating.text]
            , 'period': [period.text]
            , 'wind': [wind.find('text').text]
            , 'wind_state': [wind_state.text]
            , 'wave_height': [wave_height.find('text').text]
            , 'wave_direction': [wave_height.find('text').text]
            , 'latitude': [latitude]
            , 'longitude': [longitude]
        })], ignore_index=True)    

    print(cur_break_df)
    return cur_break_df

for index, row in pages_df.iterrows():
    # If this were in production Remove code until HERE and timestamp each pull
    if (df['break_name'] == row['break_name']).sum() > 40:
        continue
    print(row['break_name'])
    # HERE

    df = pd.concat([df, scrape(row)], ignore_index=True)

    print(scrape(row))
    data = scrape(row)

    csv_name = 'breaks_temp2.csv'
    if not os.path.isfile(csv_name):
        data.to_csv(csv_name)
    else:
        data.to_csv(csv_name, mode='a', index=True, header=False)

In [170]:
import psycopg2


try:
    connection = psycopg2.connect(user="avossmeyer",
                                  password="surfbro1#",
                                  host="surf-forecasts.c6bioghb9ybm.us-east-2.rds.amazonaws.com",
                                  port="5432",
                                  database="postgres")
    cursor = connection.cursor()

#     postgres_insert_query = """ INSERT INTO mobile (ID, MODEL, PRICE) VALUES (%s,%s,%s)"""
#     record_to_insert = (5, 'One Plus 6', 950)
#     cursor.execute(postgres_insert_query, record_to_insert)
    cursor.execute(""" SELECT * FROM airports limit 100""")
    
    row = cursor.fetchone()
    print(row)


    connection.commit()
    count = cursor.rowcount
    print(count, "Record inserted successfully into mobile table")

except (Exception, psycopg2.Error) as error:
    print("Failed to insert record into mobile table", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


('AYGA', 'GKA', 'GOROKA', 'GOROKA', 'PAPUA NEW GUINEA', 6, 4, 54, 'S', 145, 23, 30, 'E', 1610, -6.082, 145.392, 1)
100 Record inserted successfully into mobile table
PostgreSQL connection is closed
